In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + "0.0.32.post2" if v == "2.8.0" else "0.0.29.post3"
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length =
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/Qwen2.5-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,)

Unsloth 2025.8.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/sample_data/test.csv")
df =df.dropna()


dataset = Dataset.from_pandas(df)

EOS_TOKEN = tokenizer.eos_token
df["text"] = "Question: " + df["question"] + "\nSQL: " + df["sql"] + EOS_TOKEN
dataset = Dataset.from_pandas(df[["text"]])


In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/8280 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,280 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


Step,Training Loss
1,3.470700
2,3.420500
3,3.637600
4,3.428600
5,3.321800
6,2.460500
7,2.547300
8,2.255400
9,2.024100
10,2.220100


In [10]:
trainer.model.save_pretrained("/content/outputs")
trainer.tokenizer.save_pretrained("/content/outputs")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('/content/outputs/tokenizer_config.json',
 '/content/outputs/special_tokens_map.json',
 '/content/outputs/vocab.json',
 '/content/outputs/merges.txt',
 '/content/outputs/added_tokens.json',
 '/content/outputs/tokenizer.json')

In [59]:
!pip install peft
!pip install -U langchain langchain-community
!pip install langchain-huggingface
!pip install langgraph


from langchain_core.tools import tool
from peft import PeftModel
from __future__ import annotations
from langchain_community.utilities import SQLDatabase
import os, time, requests, mimetypes
from mcp.server.fastmcp import FastMCP
from dotenv import load_dotenv
import pandas as pd
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLDataBaseTool,
)
from datetime import datetime
from sqlalchemy import create_engine
from langchain_core.messages import AIMessage, AIMessageChunk, SystemMessage
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from transformers import pipeline
from unsloth.chat_templates import get_chat_template


db = SQLDatabase.from_uri("sqlite:////content/sample_data/receipts.db")


BASE = "unsloth/Qwen2.5-1.5B"
base_model, tokenizer = FastLanguageModel.from_pretrained(
    BASE,
    max_seq_length=2048,
    load_in_4bit=True,
)

tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")

model = PeftModel.from_pretrained(base_model, "/content/outputs")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
)


@tool
def get_current_date() -> str:
    """Get the current date in YYYY-MM-DD format."""
    return datetime.now().strftime("%Y-%m-%d")

@tool
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

tools = [get_current_date, list_tables, tables_schema]

def create_query_agent():

  llm = HuggingFacePipeline(pipeline=pipe)
  chat_llm = ChatHuggingFace(llm = llm,tokenizer= tokenizer)
  chat_llm_with_tools = chat_llm.bind_tools(tools)
  return chat_llm_with_tools




==((====))==  Unsloth 2025.8.9: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0


In [63]:

from langgraph.prebuilt import ToolNode
from langchain_core.messages import BaseMessage, AIMessage, SystemMessage, HumanMessage


system_message =  SystemMessage("""You are a SQL expert with strong attention to detail.

    You have FOUR tools:
    - list_tables(): lists all tables
    - tables_schema(tables: str): takes a comma-separated list of table names and returns their schema and sample rows
    - execute_sql(sql_query: str): executes a single SELECT query and returns results
    - get_current_date(): returns the current date in YYYY-MM-DD format just so youre always in the loop of what today is

    MANDATORY WORKFLOW:
    1) ALWAYS call list_tables() first to see available tables.
    2) THEN call tables_schema() ONLY for tables that seem relevant to the question.
    3) Based on the schema, construct ONE safe SELECT query (no DML/DDL). Never select *; only needed columns.
    4) Call execute_sql() with that query.
    5) If execute_sql fails or returns empty, refine the query and try again (loop at most 3 times).
    6) Then answer the user in plain text.

    RULES:
    - Do not ask the user for table or column names unless you have already called list_tables() and tables_schema()
    - Always query both tables for optimal results
    - LIMIT results to at most 5 rows unless the user requested another number or show all.
    - Never perform INSERT/UPDATE/DELETE/DROP/ALTER/etc.
    - "When querying for an item or a specific vendor or a specific category the name may vary: "
        "FIRST run: SELECT DISTINCT Item FROM receipts or receipt_items "
        "WHERE lower(Item) LIKE '%milk%'; "
        "Then choose the best match and run a precise SELECT."

    """)

model = create_query_agent()
query = "How much did I spend at restaurants?"
conversation = [system_message, HumanMessage(content=query)]

response = model.invoke(conversation)
conversation.append(response)

while getattr(response, "tool_calls", None):
    tool_state = await tool_node.ainvoke({"messages": [response]})
    tool_messages = tool_state.get("messages", [])
    conversation.extend(tool_messages)

    response = model.invoke(conversation)
    conversation.append(response)

final_answer = response.content
print(final_answer)



<|im_start|>system
You are a SQL expert with strong attention to detail.

    You have FOUR tools:
    - list_tables(): lists all tables
    - tables_schema(tables: str): takes a comma-separated list of table names and returns their schema and sample rows
    - execute_sql(sql_query: str): executes a single SELECT query and returns results
    - get_current_date(): returns the current date in YYYY-MM-DD format just so youre always in the loop of what today is 

    MANDATORY WORKFLOW:
    1) ALWAYS call list_tables() first to see available tables.
    2) THEN call tables_schema() ONLY for tables that seem relevant to the question.
    3) Based on the schema, construct ONE safe SELECT query (no DML/DDL). Never select *; only needed columns.
    4) Call execute_sql() with that query.
    5) If execute_sql fails or returns empty, refine the query and try again (loop at most 3 times).
    6) Then answer the user in plain text.

    RULES:
    - Do not ask the user for table or column names